## 6.6 データの結合

### 6.6.1 concat関数

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.notebook_repr_html', False)

adams_act = pd.DataFrame(
                [
                  ['20230102', 92],
                  ['20230103', 65],
                  ['20230104', 96]
                ],
                   columns=['Date', 'Act'])

adams_breath = pd.DataFrame(
                [
                  ['20230102', 90],
                  ['20230103', 71],
                  ['20230104', 81]
                ],
                   columns=['Date', 'Breath'])
print(adams_act)

       Date  Act
0  20230102   92
1  20230103   65
2  20230104   96


In [2]:
print(adams_breath)

       Date  Breath
0  20230102      90
1  20230103      71
2  20230104      81


In [3]:
pd.concat([adams_act, adams_breath])

       Date   Act  Breath
0  20230102  92.0     NaN
1  20230103  65.0     NaN
2  20230104  96.0     NaN
0  20230102   NaN    90.0
1  20230103   NaN    71.0
2  20230104   NaN    81.0

In [4]:
pd.concat([adams_act, adams_breath], axis=1)

       Date  Act      Date  Breath
0  20230102   92  20230102      90
1  20230103   65  20230103      71
2  20230104   96  20230104      81

In [5]:
adams_act.index

RangeIndex(start=0, stop=3, step=1)

In [6]:
new_pd = pd.concat([adams_act, adams_breath], axis=1)
new_pd

       Date  Act      Date  Breath
0  20230102   92  20230102      90
1  20230103   65  20230103      71
2  20230104   96  20230104      81

In [7]:
# 重複した列の削除
new_pd.loc[:,~new_pd.columns.duplicated()]

       Date  Act  Breath
0  20230102   92      90
1  20230103   65      71
2  20230104   96      81

In [8]:
adams_act_with_idx = adams_act.set_index('Date')
adams_breath_with_idx = adams_breath.set_index('Date')
pd.concat([adams_act_with_idx, adams_breath_with_idx], axis=1)

          Act  Breath
Date                 
20230102   92      90
20230103   65      71
20230104   96      81

In [9]:
adams_act.merge(adams_breath, left_on="Date", right_on="Date")

       Date  Act  Breath
0  20230102   92      90
1  20230103   65      71
2  20230104   96      81

In [10]:
pd.concat([adams_act, adams_breath], sort=True)

    Act  Breath      Date
0  92.0     NaN  20230102
1  65.0     NaN  20230103
2  96.0     NaN  20230104
0   NaN    90.0  20230102
1   NaN    71.0  20230103
2   NaN    81.0  20230104

In [11]:
pd.concat([adams_act, adams_breath], ignore_index=True)

       Date   Act  Breath
0  20230102  92.0     NaN
1  20230103  65.0     NaN
2  20230104  96.0     NaN
3  20230102   NaN    90.0
4  20230103   NaN    71.0
5  20230104   NaN    81.0

In [12]:
pd.concat([adams_act, adams_breath], ignore_index=True).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    6 non-null      object 
 1   Act     3 non-null      float64
 2   Breath  3 non-null      float64
dtypes: float64(2), object(1)
memory usage: 272.0+ bytes


In [13]:
pd.concat([adams_act, adams_breath], verify_integrity=False)

       Date   Act  Breath
0  20230102  92.0     NaN
1  20230103  65.0     NaN
2  20230104  96.0     NaN
0  20230102   NaN    90.0
1  20230103   NaN    71.0
2  20230104   NaN    81.0

In [14]:
pd.concat([adams_act, adams_breath], verify_integrity=True)

ValueError: Indexes have overlapping values: Int64Index([0, 1, 2], dtype='int64')

In [15]:
pd.concat([adams_act, adams_breath], keys=['level1', 'level2'],
          names=['name1', 'name2'])

                  Date   Act  Breath
name1  name2                        
level1 0      20230102  92.0     NaN
       1      20230103  65.0     NaN
       2      20230104  96.0     NaN
level2 0      20230102   NaN    90.0
       1      20230103   NaN    71.0
       2      20230104   NaN    81.0

In [16]:
pd.concat([adams_act, adams_breath], keys=['level1', 'level2'],
          levels=[['level1', 'level2']])

              Date   Act  Breath
level1 0  20230102  92.0     NaN
       1  20230103  65.0     NaN
       2  20230104  96.0     NaN
level2 0  20230102   NaN    90.0
       1  20230103   NaN    71.0
       2  20230104   NaN    81.0

In [17]:
pd.concat([adams_act, adams_breath], keys=['level1', 'level2'],
          levels=[['level1', 'level2']], names=['name1', 'name2'])

                  Date   Act  Breath
name1  name2                        
level1 0      20230102  92.0     NaN
       1      20230103  65.0     NaN
       2      20230104  96.0     NaN
level2 0      20230102   NaN    90.0
       1      20230103   NaN    71.0
       2      20230104   NaN    81.0

### 6.6.2 append メソッド

In [18]:
adams_act.append(adams_breath)

C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\3023122094.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  adams_act.append(adams_breath)


       Date   Act  Breath
0  20230102  92.0     NaN
1  20230103  65.0     NaN
2  20230104  96.0     NaN
0  20230102   NaN    90.0
1  20230103   NaN    71.0
2  20230104   NaN    81.0

#### appendとconcatのパフォーマンスの比較

In [19]:
# appendとconcatのパフォーマンスの比較
import pandas as pd
import numpy as np
import timeit

EXE_CNT = 20

df = pd.DataFrame({
    "A": np.random.randint(0, 100, 1_000_000),
    "B": np.random.randint(0, 100, 1_000_000),
    "C": np.random.randint(0, 100, 1_000_000)
})

def create_df_append():
  df2 = df
  for i in range(EXE_CNT-1):
    df2 = df2.append(df)

def create_df_concat():
  df2 = pd.concat([df]*EXE_CNT)

time1 = timeit.timeit(create_df_append, number=10)
time2 = timeit.timeit(create_df_concat, number=10)

print("Running time of append function: ", time1)
print("Running time of concat function: ", time2)

C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is

C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is

Running time of append function:  7.516886399999748
Running time of concat function:  0.6663851000002978


In [20]:
%load_ext memory_profiler
%memit create_df_append()

C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
C:\Users\sunin\AppData\Local\Temp\ipykernel_17060\239426361.py:17: FutureWarning: The frame.append method is

peak memory: 716.98 MiB, increment: 593.17 MiB


In [21]:
%memit create_df_concat()

peak memory: 123.88 MiB, increment: 0.00 MiB


### 6.6.3 insert メソッド

In [22]:
import pandas as pd
import numpy as np

adams_act = pd.DataFrame(
                [
                  ['20230102', 92],
                  ['20230103', 65],
                  ['20230104', 96]
                ],
                columns=['Date', 'Act'])
print(adams_act)

       Date  Act
0  20230102   92
1  20230103   65
2  20230104   96


In [23]:
# 二番目の列に[11, 12, 13]値の「Date2」列を追加
adams_act.insert(1, "Date2", [11, 12, 13] )
adams_act

       Date  Date2  Act
0  20230102     11   92
1  20230103     12   65
2  20230104     13   96

In [24]:
# 三番目の列に[21, 22, 23]値の「Date2」列を追加
adams_act.insert(2, "Date2", [21, 22, 23] )

ValueError: cannot insert Date2, already exists

In [25]:
# 重複列「Date2」を許可
adams_act.insert(2, "Date2", [21, 22, 23], allow_duplicates=True )
adams_act

       Date  Date2  Date2  Act
0  20230102     11     21   92
1  20230103     12     22   65
2  20230104     13     23   96

### 6.6.4 merge 関数とメソッド

In [26]:
pd.set_option('display.notebook_repr_html', False)

# 再定義
adams_act = pd.DataFrame(
                [
                  ['20230102', 92],
                  ['20230103', 65],
                  ['20230104', 96]
                ],
                columns=['Date', 'Act'])

adams_breath = pd.DataFrame(
                [
                  ['20230102', 90],
                  ['20230103', 71],
                  ['20230104', 81]
                ],
                columns=['Date', 'Breath'])

pd.merge(adams_act,adams_breath)

       Date  Act  Breath
0  20230102   92      90
1  20230103   65      71
2  20230104   96      81

In [27]:
# 多対多結合
adams_act2 = pd.DataFrame(
                [
                  ['20230102', 92],
                  ['20230102', 86],
                  ['20230103', 65],
                  ['20230104', 96]
                ],
                columns=['Date', 'Act'])
adams_breath2 = pd.DataFrame(
                [
                  ['20230102', 90],
                  ['20230102', 89],
                  ['20230103', 71],
                  ['20230104', 81]
                ],
                columns=['Date', 'Breath'])
pd.merge(adams_act2, adams_breath2)

       Date  Act  Breath
0  20230102   92      90
1  20230102   92      89
2  20230102   86      90
3  20230102   86      89
4  20230103   65      71
5  20230104   96      81

#### validateパラメータ

In [28]:
# validate
pd.merge(adams_act2, adams_breath2, validate='1:m')

MergeError: Merge keys are not unique in left dataset; not a one-to-many merge

In [29]:
pd.merge(adams_act2, adams_breath2, validate='m:m')

       Date  Act  Breath
0  20230102   92      90
1  20230102   92      89
2  20230102   86      90
3  20230102   86      89
4  20230103   65      71
5  20230104   96      81

#### indicatorパラメータ

In [30]:
# indicator
pd.merge(adams_act2, adams_breath2, indicator=True)

       Date  Act  Breath _merge
0  20230102   92      90   both
1  20230102   92      89   both
2  20230102   86      90   both
3  20230102   86      89   both
4  20230103   65      71   both
5  20230104   96      81   both

#### onパラメータ

In [31]:
import numpy as np
df4_1 = pd.DataFrame({'I1': ['a', 'b', 'c'], 'I2': ['x', 'y', 'z'],
                      'V1': range(3) })
df4_2 = pd.DataFrame({'I1': ['a', 'b', 'c'], 'I2': ['x', 'y', 'z'],
                      'V4': np.random.randint(10,20, 3) })
pd.merge(df4_1, df4_2, indicator=True)

  I1 I2  V1  V4 _merge
0  a  x   0  14   both
1  b  y   1  11   both
2  c  z   2  18   both

In [32]:
import numpy as np
# 'I2'の値を変更
df4_1 = pd.DataFrame({'I1': ['a', 'b', 'c'], 'I2': ['d', 'e', 'z'],
                      'V1': range(3) })
# 'I1'の値を変更
df4_2 = pd.DataFrame({'I1': ['d', 'e', 'f'], 'I2': ['x', 'y', 'z'],
                      'V4': np.random.randint(10,20, 3) })
pd.merge(df4_1, df4_2, indicator=True)

Empty DataFrame
Columns: [I1, I2, V1, V4, _merge]
Index: []

In [33]:
pd.merge(df4_1, df4_2, indicator=True).info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   I1      0 non-null      object  
 1   I2      0 non-null      object  
 2   V1      0 non-null      int64   
 3   V4      0 non-null      int32   
 4   _merge  0 non-null      category
dtypes: category(1), int32(1), int64(1), object(2)
memory usage: 132.0+ bytes


In [34]:
import numpy as np
df4_1 = pd.DataFrame({'I1': ['a', 'b', 'c'], 'I2': ['d', 'e', 'z'],
                      'V1': range(3) })
df4_2 = pd.DataFrame({'I1': ['d', 'e', 'f'], 'I2': ['x', 'y', 'z'],
                      'V4': np.random.randint(10,20, 3) })
pd.merge(df4_1, df4_2, on="I2", indicator=True)

  I1_x I2  V1 I1_y  V4 _merge
0    c  z   2    f  19   both

#### how, left_on, right_onパラメータ

In [35]:
# how, left_on, right_onパラメータ
import numpy as np
df4_1 = pd.DataFrame({'I1': ['a', 'b', 'c'], 'I2': ['d', 'e', 'z'],
                      'V1': range(3) })
df4_2 = pd.DataFrame({'I1': ['d', 'e', 'f'], 'I2': ['x', 'y', 'z'],
                      'V4': np.random.randint(10,20,3) })
pd.merge(df4_1, df4_2, left_on="I2", right_on="I1", indicator=True)

  I1_x I2_x  V1 I1_y I2_y  V4 _merge
0    a    d   0    d    x  12   both
1    b    e   1    e    y  19   both

In [36]:
df1 = pd.DataFrame({'I1': ['a', 'b', 'c'], 'V1': range(3) })
df2 = pd.DataFrame({'I2': ['a', 'b', 'd'], 'V2': range(3) })
print("==== df1 ==== ")
print(df1)
print("\n ==== df2 ====")
print(df2)

==== df1 ==== 
  I1  V1
0  a   0
1  b   1
2  c   2

 ==== df2 ====
  I2  V2
0  a   0
1  b   1
2  d   2


In [37]:
# inner 内部結合
pd.merge(df1, df2, how="inner", left_on="I1", right_on="I2")

  I1  V1 I2  V2
0  a   0  a   0
1  b   1  b   1

In [38]:
# outer, 外部結合
pd.merge(df1, df2, how="outer", left_on="I1", right_on="I2")

    I1   V1   I2   V2
0    a  0.0    a  0.0
1    b  1.0    b  1.0
2    c  2.0  NaN  NaN
3  NaN  NaN    d  2.0

In [39]:
# cross, 両オブジェクトのデカルト積結合
pd.merge(df1, df2, how="cross")

  I1  V1 I2  V2
0  a   0  a   0
1  a   0  b   1
2  a   0  d   2
3  b   1  a   0
4  b   1  b   1
5  b   1  d   2
6  c   2  a   0
7  c   2  b   1
8  c   2  d   2

In [40]:
# left, 左側のオブジェクトに右側のオブジェクトを合わせる
pd.merge(df1, df2, how="left", left_on="I1", right_on="I2")

  I1  V1   I2   V2
0  a   0    a  0.0
1  b   1    b  1.0
2  c   2  NaN  NaN

#### suffixesパラメータ

In [41]:
# suffixes
df1 = pd.DataFrame({'I1': ['a', 'b', 'c'], 'V1': range(3) })
df3 = pd.DataFrame({'I3': ['a', 'b', 'd'], 'V1': range(3),
                    'V3': range(3) })
pd.merge(df1, df3, how="left", left_on="I1", right_on="I3",
         suffixes=('_l', '_r'), indicator=True)

  I1  V1_l   I3  V1_r   V3     _merge
0  a     0    a   0.0  0.0       both
1  b     1    b   1.0  1.0       both
2  c     2  NaN   NaN  NaN  left_only

#### left_index, right_indexパラメータ

In [42]:
# left_index, right_indexパラメータ
left_df = pd.DataFrame([[11, 12], [13, 14], [15,16]],
                       index=['a', 'b', 'c'],
                       columns=['val1', 'val2'])
left_df

   val1  val2
a    11    12
b    13    14
c    15    16

In [43]:
right_df = pd.DataFrame([[21, 22], [23, 24], [25, 26]],
                        index=['c', 'd', 'e'],
                        columns=['val1', 'val2'])
right_df

   val1  val2
c    21    22
d    23    24
e    25    26

In [44]:
# 単純に行に結合
pd.merge(left_df, right_df, how="outer")

   val1  val2
0    11    12
1    13    14
2    15    16
3    21    22
4    23    24
5    25    26

In [45]:
# ２つのオブジェクトのインデクスを基準に結合
pd.merge(left_df, right_df, how="outer",
         left_index=True, right_index=True)

   val1_x  val2_x  val1_y  val2_y
a    11.0    12.0     NaN     NaN
b    13.0    14.0     NaN     NaN
c    15.0    16.0    21.0    22.0
d     NaN     NaN    23.0    24.0
e     NaN     NaN    25.0    26.0

### 6.6.5 joinメソッド

In [46]:
import pandas as pd
import numpy as np
left = pd.DataFrame({'idx': ['a', 'b', 'c', 'd'],
                     'val': ['A1', 'A2', 'A3', 'A4']})
right = pd.DataFrame({'idx': ['a', 'b', 'd'],
                      'val': ['R1', 'R2', 'R4']})
print(left)

  idx val
0   a  A1
1   b  A2
2   c  A3
3   d  A4


In [47]:
print(right)

  idx val
0   a  R1
1   b  R2
2   d  R4


In [48]:
# もしインデックス以外の列名が同じであれば、エラーが発生します。
left.join(right)

ValueError: columns overlap but no suffix specified: Index(['idx', 'val'], dtype='object')

In [49]:
# 接尾辞を付けてみましょう。
left.join(right, lsuffix="_l", rsuffix="_r")

  idx_l val_l idx_r val_r
0     a    A1     a    R1
1     b    A2     b    R2
2     c    A3     d    R4
3     d    A4   NaN   NaN

In [50]:
# 2つのオブジェクトに明示的なインデックスを設定しましょう。
left.set_index('idx', inplace=True)
right.set_index('idx', inplace=True)

In [51]:
# 再度joinしてみましょう。
left.join(right, lsuffix="_l", rsuffix="_r")

    val_l val_r
idx            
a      A1    R1
b      A2    R2
c      A3   NaN
d      A4    R4

### 6.6.6 combineメソッド

In [52]:
import pandas as pd
import numpy as np
# df1 = pd.DataFrame({'col1': np.random.randint(0, 10, 3),
#                     'col2': np.random.randint(30, 40, 3)})
# df2 = pd.DataFrame({'col1': np.random.randint(10, 20, 3),
#                    'col2': np.random.randint(20,30, 3)})
df1 = pd.DataFrame({'col1': [7,4,5],
                    'col2': [38,30,35]})
df2 = pd.DataFrame({'col1': [15,15,18],
                   'col2': [29,25,21]})
print(df1)

   col1  col2
0     7    38
1     4    30
2     5    35


In [53]:
print(df2)

   col1  col2
0    15    29
1    15    25
2    18    21


In [54]:
df1.combine(df2, np.maximum)

   col1  col2
0    15    38
1    15    30
2    18    35

In [55]:
# 同じかより大きい値を返す
def get_greater_equals(v1, v2):
  return v1 if v1.sum() >= v2.sum() else v2

df1.combine(df2, get_greater_equals)

   col1  col2
0    15    38
1    15    30
2    18    35

In [56]:
import pandas as pd
import numpy as np
# df1 = pd.DataFrame({'col1': np.random.randint(0, 10, 3),
#                     'col2': np.random.randint(30, 40, 3)})
# df2 = pd.DataFrame({'col1': np.random.randint(10, 20, 3),
#                     'col2': np.array([21, np.nan, np.nan])})
df1 = pd.DataFrame({'col1': [9,3,9],
                    'col2': [30,35,38]})
df2 = pd.DataFrame({'col1': [10,13,10],
                    'col2': np.array([21, np.nan, np.nan])})

print(df1)

   col1  col2
0     9    30
1     3    35
2     9    38


In [57]:
print(df2)

   col1  col2
0    10  21.0
1    13   NaN
2    10   NaN


In [58]:
# 同じかより小さい値を返す。
def get_less_equals(v1, v2):
  return v1 if v1.sum() <= v2.sum() else v2

# 結合する前に欠損値を-10に置き換える。
df1.combine(df2, get_less_equals, fill_value=-10)

   col1  col2
0     9  21.0
1     3 -10.0
2     9 -10.0

In [59]:
df1 = pd.DataFrame({'col1': np.array([1, 2, 3]),
                    'col3': np.array([31,32,33])})
df3 = pd.DataFrame({'col1': np.array([11, 12, 13]) })
df1

   col1  col3
0     1    31
1     2    32
2     3    33

In [60]:
df3

   col1
0    11
1    12
2    13

In [61]:
# overwrite=Trueの場合、selfの列がotherにない場合、NaNで置き換えます。
print(df1.combine(df3, np.maximum,overwrite=True))

   col1  col3
0    11   NaN
1    12   NaN
2    13   NaN


In [62]:
print(df1.combine(df3, np.maximum,overwrite=False))

   col1  col3
0    11    31
1    12    32
2    13    33


### 6.6.7 combine_firstメソッド

In [63]:
import pandas as pd
import numpy as np
df1 = pd.DataFrame({'col1': [np.nan, np.nan, 1],
                    'col2': [np.nan, 11, 12]})
df2 = pd.DataFrame({'col1': [21, 22, 23],
                    'col2': [np.nan, 31, 32]})
df1

   col1  col2
0   NaN   NaN
1   NaN  11.0
2   1.0  12.0

In [64]:
df2

   col1  col2
0    21   NaN
1    22  31.0
2    23  32.0

In [65]:
df1.combine_first(df2)

   col1  col2
0  21.0   NaN
1  22.0  11.0
2   1.0  12.0

### 6.6.8 updateメソッド

In [66]:
import pandas as pd
import numpy as np
df1 = pd.DataFrame({'col1': [np.nan, np.nan, 1],
                    'col2': [np.nan, 11, 12]})
df2 = pd.DataFrame({'col1': [21, 22, 23],
                    'col2': [np.nan, 31, 32]})
print(df1)

   col1  col2
0   NaN   NaN
1   NaN  11.0
2   1.0  12.0


In [67]:
print(df2)

   col1  col2
0    21   NaN
1    22  31.0
2    23  32.0


In [68]:
df1.update(df2)
df1

   col1  col2
0  21.0   NaN
1  22.0  31.0
2  23.0  32.0

In [69]:
df1 = pd.DataFrame({'col1': [np.nan, np.nan, 1],
                    'col2': [np.nan, 11, 12]} )
df2 = pd.DataFrame({'col1': [21, 22, 23],
                    'col2': [np.nan, 31, 32]} )
print(df1)

   col1  col2
0   NaN   NaN
1   NaN  11.0
2   1.0  12.0


In [70]:
print(df2)

   col1  col2
0    21   NaN
1    22  31.0
2    23  32.0


In [71]:
df1.update(df2, overwrite=False)
df1

   col1  col2
0  21.0   NaN
1  22.0  11.0
2   1.0  12.0

In [72]:
df1 = pd.DataFrame({'col1': [np.nan, np.nan, 1],
                    'col2': [np.nan, 11, 12]})
df2 = pd.DataFrame({'col1': [21, 22, 23],
                    'col2': [np.nan, 31, 32]})
df1.update(df2, filter_func=lambda x: x < 12)
df1

   col1  col2
0   NaN   NaN
1   NaN  31.0
2  23.0  12.0

In [73]:
df1 = pd.DataFrame({'col1': [np.nan, np.nan, 1],
                    'col2': [np.nan, np.nan, 12]} )
df2 = pd.DataFrame({'col1': [21, 22, np.nan],
                    'col2': [np.nan, 31, 32]} )
df1.update(df2, errors="raise")

ValueError: Data overlaps.

In [74]:
df1 = pd.DataFrame({'col1': [np.nan, np.nan, 1],
                    'col2': [np.nan, np.nan, 12]} )
df2 = pd.DataFrame({'col1': [21, 22, np.nan],
                    'col2': [np.nan, 31, np.nan]} )
df1.update(df2, errors="raise")
df1

   col1  col2
0  21.0   NaN
1  22.0  31.0
2   1.0  12.0